In [ ]:
#Packages import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def volts(IVCurve): #input: csv file containing IV curve. Returns Vp, Vf
    IV = pd.read_csv(IVCurve).to_numpy()
    print(IV)
    

In [ ]:
volts("LP/Trace_10p8mTorr_400W_1.csv")

3


C:\Users\Ahack\AppData\Local\Temp\ipykernel_3952\3635653937.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(int(np.argwhere(x >5)[0]))
